In [1]:
import pandas as pd
import re
import subprocess

In [2]:
command_devices = "adb devices"
print(subprocess.check_output(command_devices, shell=True))

command = "adb shell dumpsys usagestats > usagestats.log"
subprocess.run(command, shell=True, check=True)

b'List of devices attached\r\n143028c2\tdevice\r\n\r\n'


CompletedProcess(args='adb shell dumpsys usagestats > usagestats.log', returncode=0)

In [3]:
# 打开文件并读取内容，指定编码方式为"utf-8"
with open('usagestats.log', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 找到包含"Last 24 hour events"和"In-memory daily stats"的行，获取这两行之间的所有行
start_index = next(i for i, line in enumerate(lines) if 'Last 24 hour events' in line) + 1
end_index = next(i for i, line in enumerate(lines) if 'In-memory daily stats' in line)
selected_lines = lines[start_index:end_index]

In [4]:
# 初始化DataFrame
df = pd.DataFrame(columns=['time', 'type', 'package'])

# 对每一行使用正则表达式匹配time、type和package的值，并将匹配的值添加到DataFrame中
for line in selected_lines:
    match = re.search(r'time="(.*?)" type=(.*?) package=(.*?) ', line)
    if match:
        new_row = {'time': match.group(1), 'type': match.group(2), 'package': match.group(3)}
        df.loc[len(df)] = new_row

In [5]:
# 将DataFrame保存为csv文件
df.to_csv('usagestats.csv', index=False, encoding='utf-8')